# Unit 7 lab : Map/Reduce

*You should carry out this lab using Anaconda on either a University lab desktop or your own laptop. (If you use Colab, you will only have one core, and then you wouldn't be able to get multiple map or reduce processes actually carrying out parallel computation.) In case you still haven't downloaded Anaconda, see the instructions under Getting Started in Canvas.*



The following code loads in the text of H.G.Wells' novel "Time Machine" and turns the novel into a list of lines and each line into a list of tokens.  We will think of and refer to each line as a *document*.  Run the code and print out the first five lines.

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize


document1="timemachine.txt"

lines=[]
with open(document1) as instream:
    for line in instream:        
        tokens=[token.lower() for token in word_tokenize(line.rstrip())]  #strip excess white space, tokenize, lower-case
        lines.append(tokens)
        
        

If this has an error, you probably need to run the nltk downloader on your machine to get the necessary resources. If this happens, then run the following code.

In [ ]:
import nltk
nltk.download('punkt')

### Exercise 1
Write a simple function (i.e., without using map/reduce) to count the number of times each word occurs and store it in a dictionary.  Use it to find out how many times the word 'time' occurs in the novel.


Now lets see how we can rewrite this in the Map/Reduce paradigm.  Below is a map_reduce function which takes a list of inputs, a mapper function and a reducer function

In [ ]:
from collections import defaultdict

def map_reduce(inputs,mapper,reducer):
    
    collector=defaultdict(list)  #this dictionary is where we will store intermediate results
                                 #it will map keys to lists of values (default value of a list is [])
                                 #in a real system, this would be stored in individual files at the map nodes
                                 #and then transferred to the reduce nodes
    
    #map stage
    for input in inputs:
        for key, value in mapper(input):     #pass each input to the mapper function and receive back each key,value pair yielded
            collector[key].append(value)     #append the value to the list for that key in the intermediate store
            
    #reduce stage - 1 reducer for each key
    outputs=[]
    for key,values in collector.items():  #take each pair of key, value-lists and pass it to the reduce function
        for res in reducer(key,values):       #take each pair yielded by the reducer and add it to the outputs list
            outputs.append(res)
    
    return outputs


Now we define a mapper function and a reducer function particular to our word-counting task

In [ ]:
def wc_mapper(document):
    for word in document:
        yield(word,1)
        
def wc_reducer(word,counts):
    yield(word,sum(counts))
        

Lets run this and display our outputs


In [ ]:
#display() just prints the dictionary items using tab separation
#ipl is the desired number of columns per line
def display(results,ipl=4):
    line=""
    itemcount=1
    for res in results:
        for thing in res:
            if itemcount<ipl:
                line+=str(thing)+"\t"
                itemcount+=1
            else:
                line+=str(thing)
                print(line)
                itemcount=1
                line=""
                
results = map_reduce(lines,wc_mapper,wc_reducer)

display(results)

### Exercise 2
Turn the output of map_reduce (i.e., results) into a Pandas dataframe  or a dictionary.  Then
1. look up the number of times 'time' occurs.
2. find all of the words which occur more than 200 times

Here is some code which uses a pool of processes to do the map tasks and reduce tasks.  A slight modification is needed for the map and reduce functions so that they return lists rather than the 'unpickleable' generators.


In [ ]:
import operator
from multiprocessing import Pool

def map_reduce_parallel(inputs,mapper,reducer,mapprocesses=3,reduceprocesses=2):
    
    collector=defaultdict(list)  #this dictionary is where we will store intermediate results
                                 #it will map keys to lists of values (default value of a list is [])
                                 #in a real system, this would be stored in individual files at the map nodes
                                 #and then transferred to the reduce nodes
    
    mappool = Pool(processes=mapprocesses)
    #map stage
    
    mapresults=mappool.map(mapper,inputs)
    mappool.close()
    #print mapresult
    for mapresult in mapresults:
        for (key, value) in mapresult:     #pass each input to the mapper function and receive back each key,value pair yielded
            collector[key].append(value)     #append the value to the list for that key in the intermediate store
            
    #reduce stage 
    outputs=[]
    reducepool = Pool(processes=reduceprocesses)
    
    reduceresults=reducepool.map(reducer,collector.items())
    reducepool.close()
    for reduceresult in reduceresults:
        outputs+=reduceresult
   
    return outputs


If you get an error because multiprocess is not installed, then create a new cell below with the following command: *pip install multiprocess*

In [ ]:
import defs  # auxiliary python program with the map and reduce functions: defs.py 
             
# Here are these functions, commented out, for your information:

# def wc_mapper_parallel(document):
#     output=[]
#     for word in document:
#         output.append((word,1))
        
#     return output

# def wc_reducer_parallel(item):
#     output=[]
#     (word,counts)=item
#     output.append((word,sum(counts)))
#     return output
        
results=map_reduce_parallel(lines,defs.wc_mapper_parallel,defs.wc_reducer_parallel)
#results
df=pd.DataFrame(results,columns=["word","count"])
df[df["word"]=="time"]

### Exercise 3
Time the performance of the parallel version of map_reduce with different numbers of map and reduce processes.  
* What is the best number of processes for each stage on the computer you are working on?
* How fast is the original non-parallel version of map_reduce?  
* Explain why this is significantly different to the parallel version (with 1 process at each stage).

### Exercise 4
Write some map and reduce functions of your own.  Maybe ...
1. the frequency of each length of word
2. the mean length of a word starting with each possible character
3. matrix-vector multiplication
4. matrix-matrix multiplication